In [10]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image
import glob
import shutil


# Data PreProcessing

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# Hyperparameters
num_classes = 200           # Tiny ImageNet has 200 classes
batch_size = 32
num_epochs = 10
learning_rate = 0.001

cpu


In [3]:
train_transforms = transforms.Compose([
    transforms.Resize((256, 256)), 
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])
train_dataset = datasets.ImageFolder('./tiny-imagenet-200/train', transform=train_transforms)

In [4]:
val_dir = './tiny-imagenet-200/val'
annotation_file = os.path.join(val_dir, 'val_annotations.txt')
new_val_dir = './tiny-imagenet-200/val-images-by-class'
os.makedirs(new_val_dir, exist_ok=True)

with open(annotation_file, 'r') as f:
    for line in f:
        parts = line.strip().split('\t')
        img_name, class_name = parts[0], parts[1]
        class_dir = os.path.join(new_val_dir, class_name)
        os.makedirs(class_dir, exist_ok=True)
        src = os.path.join(val_dir, 'images', img_name)
        dst = os.path.join(class_dir, img_name)
        shutil.copy(src, dst)

val_dataset = datasets.ImageFolder(new_val_dir, transform=train_transforms)

In [6]:
class TinyImageNetTestDataset(Dataset):
    def __init__(self, test_dir, transform=None):
        self.test_dir = test_dir
        self.transform = transform
        # List all image files in the test folder
        self.image_paths = glob.glob(os.path.join(test_dir, '*.JPEG'))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, os.path.basename(img_path)
    
test_dataset = TinyImageNetTestDataset('./tiny-imagenet-200/test', transform=train_transforms)


In [11]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

# VGG16 Model Implementation

In [9]:
# Load a pre-trained VGG16 model
model = models.vgg16(weights=True)

# Modify the final fully connected layer to match the number of classes in Tiny ImageNet
model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
model = model.to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


# Training Loop

In [ ]:
for epoch in tqdm(range(num_epochs)):
    print(f'Epoch {epoch + 1}/{num_epochs}')
    for phase, loader in zip(['train', 'val'], [train_loader, val_loader]):
        if phase == 'train':
            model.train()  # Set model to training mode
        else:
            model.eval()   # Set model to evaluation mode

        running_loss = 0.0
        running_corrects = 0

        # Iterate over data.
        for inputs, labels in loader[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()

            # Forward pass; track history only if in training phase
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                # Backward pass + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(loader.dataset)
        epoch_acc = running_corrects.double() / len(loader.dataset)
        print(f'{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
    print('-' * 30)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1/10


  0%|          | 0/10 [01:19<?, ?it/s]


KeyboardInterrupt: 

# Feature Visualization of Each Layer

In [ ]:
# Dictionary to store activations
activations = {}

# Hook function to capture the output of a layer
def get_activation(name):
    def hook(model, input, output):
        activations[name] = output.detach()
    return hook

# Register hooks on each layer in the VGG16 feature extractor (the convolutional layers)
for idx, layer in enumerate(model.features):
    layer.register_forward_hook(get_activation(f'features_{idx}'))

# Get a sample batch from the validation set (or use a single image)
model.eval()
sample_inputs, _ = next(iter(dataloaders['val']))
sample_inputs = sample_inputs.to(device)

# Forward pass to capture activations
_ = model(sample_inputs)

# Visualize the activations for each registered layer for the first image in the batch
for layer_name, activation in activations.items():
    # activation shape: [batch_size, channels, height, width]
    act = activation[0].cpu()  # Take the first image in the batch
    num_channels = act.shape[0]
    # Visualize up to 8 feature maps from the layer to keep the plots readable
    n_cols = min(num_channels, 8)
    plt.figure(figsize=(n_cols * 2, 2))
    for i in range(n_cols):
        plt.subplot(1, n_cols, i + 1)
        plt.imshow(act[i].numpy(), cmap='viridis')
        plt.title(f'{layer_name}\nChannel {i}')
        plt.axis('off')
    plt.suptitle(f'Activations from {layer_name}')
    plt.show()